In [139]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn import metrics
import numpy as npw
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler


# Read the CSV file
data = pd.read_csv("../data/data/aapl_raw_data.csv")

data = data.drop("date", axis=1)

data.isnull().sum()
data=data.fillna(0)  # Filling null values with zero
data.isnull().sum()

data = data.astype('float32')


# Keep data until 31.08.2023
data = data.iloc[:10731]

#print(data['open'].dtype)
#print(data.shape)

# Assuming 'data' is a pandas DataFrame
x_data = data[['open', 'high', 'low', 'volume', 'adjusted_close', 'change_percent', 'avg_vol_20d']]
y_data = data["close"]

# Now x_data and y_data are pandas DataFrames/Series, respectively

x_data.tail(1)

,open,high,low,volume,adjusted_close,change_percent,avg_vol_20d
10730,191.410004,192.669998,190.240005,46778000.0,190.422897,-0.59,54388236.0


In [140]:
# Set the window size for training
train_window_size = 20

# Initialize lists to store training and temporary sets
x_train_list, y_train_list, x_temp_list, y_temp_list = [], [], [], []

# Iterate through the data with the specified window size
for i in range(0, len(x_data) - train_window_size, train_window_size + 1):
    x_train_temp = x_data.iloc[i:i+train_window_size+1]
    y_train_temp = y_data.iloc[i:i+train_window_size+1]

    # Separate the last row for the temporary set
    x_train = x_train_temp.iloc[:-1]
    y_train = y_train_temp.iloc[:-1]

    x_temp = x_train_temp.iloc[-1:]
    y_temp = y_train_temp.iloc[-1:]

    x_train_list.append(x_train)
    y_train_list.append(y_train)
    x_temp_list.append(x_temp)
    y_temp_list.append(y_temp)

# Concatenate the lists into pandas DataFrames
x_train = pd.concat(x_train_list)
y_train = pd.concat(y_train_list)
x_temp = pd.concat(x_temp_list)
y_temp = pd.concat(y_temp_list)

# print(y_train.head(50))
x_temp_train, x_temp_val, y_temp_train, y_temp_val = train_test_split(x_temp, y_temp, test_size=0.2, random_state=42)


# Split x_temp and y_temp into validation and test sets
x_val, x_test, y_val, y_test = train_test_split(
    x_temp, y_temp, test_size=0.5, random_state=42)


check: last rows (x_data & x_test) match

In [141]:


"""
# Print the last 5 rows of x_data
print("Last 5 rows of x_data:")
print(x_data.tail(5))

# Print the last 5 rows of x_train
print("\nLast 25 rows of x_train:")
print(x_train.tail(25))

print("\nLast 3 rows of y_train:")
print(y_temp.tail(3))
"""

'\n# Print the last 5 rows of x_data\nprint("Last 5 rows of x_data:")\nprint(x_data.tail(5))\n\n# Print the last 5 rows of x_train\nprint("\nLast 25 rows of x_train:")\nprint(x_train.tail(25))\n\nprint("\nLast 3 rows of y_train:")\nprint(y_temp.tail(3))\n'

In [142]:
scaler = MinMaxScaler()

x_train_normalized = scaler.fit_transform(x_train)
x_val_normalized = scaler.transform(x_val)
x_test_normalized = scaler.transform(x_test)

# Convert the data to PyTorch tensors
x_train_tensor = torch.tensor(x_train_normalized, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)

x_val_tensor = torch.tensor(x_val_normalized, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32).view(-1, 1)

x_test_tensor = torch.tensor(x_test_normalized, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

check: reverse the normalization applied by the MinMaxScaler to obtain the original data

In [143]:
"""
x_test_original = scaler.inverse_transform(x_train_tensor.numpy())
print("\nFirst row of x_test_original:")
print(x_test_original[0])

print("\nFirst row of x_train:")
print(x_train.head(1))



print("\nLast row of x_test_original:")
print(x_test_original[-1])

print("\nLast row of x_train:")
print(x_train.tail(1))
"""

'\nx_test_original = scaler.inverse_transform(x_train_tensor.numpy())\nprint("\nFirst row of x_test_original:")\nprint(x_test_original[0])\n\nprint("\nFirst row of x_train:")\nprint(x_train.head(1))\n\n\n\nprint("\nLast row of x_test_original:")\nprint(x_test_original[-1])\n\nprint("\nLast row of x_train:")\nprint(x_train.tail(1))\n'

In [144]:
(len(x_train_tensor) - window_size) % batch_size == 0

False

In [145]:

# tensor typically represents: [batch_size, window_size, features]
learning_rate = 0.001
window_size = 20
test_window_size = 20
hidden_dim = 64
n_layers = 4
batch_evaluation_frequency = 10
epochs = len(x_train) - window_size
batch_size = 16
input_size = x_train.shape[1]  # Input size based on your dataset
output_size = 1  # Output size (for regression task)

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_dim, n_layers, output_size):
        super(LSTMModel, self).__init__()

        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.lstm = nn.LSTM(input_size, hidden_dim, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x, hidden):
        batch_size = x.size(0)
        out, hidden = self.lstm(x, hidden)
        out = self.fc(out[:, -1, :])
        return out, hidden

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        return hidden


    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        return hidden


'\n# tensor typically represents: [batch_size, window_size, features]\nlearning_rate = 0.001\nwindow_size = 20\ntest_window_size = 20\nhidden_dim = 64\nn_layers = 4\nbatch_evaluation_frequency = 10\nepochs = len(x_train) - window_size\nbatch_size = 16\ninput_size = x_train.shape[1]  # Input size based on your dataset\noutput_size = 1  # Output size (for regression task)\n\nclass LSTMModel(nn.Module):\n    def __init__(self, input_size, hidden_dim, n_layers, output_size):\n        super(LSTMModel, self).__init__()\n\n        self.hidden_dim = hidden_dim\n        self.n_layers = n_layers\n\n        self.lstm = nn.LSTM(input_size, hidden_dim, n_layers, batch_first=True)\n        self.fc = nn.Linear(hidden_dim, output_size)\n\n    def forward(self, x, hidden):\n        batch_size = x.size(0)\n        out, hidden = self.lstm(x, hidden)\n        out = self.fc(out[:, -1, :])\n        return out, hidden\n\n    def init_hidden(self, batch_size):\n        weight = next(self.parameters()).data\n 

In [146]:
print(f"Shape of x_train_tensor: {x_train_tensor.shape}")


Shape of x_train_tensor: torch.Size([10220, 7])


 tensor typically represents: [batch_size, window_size, features]

In [147]:
# Training loop
for epoch in range(epochs):
    # Initialize hidden state for each epoch
    hidden = model.init_hidden(batch_size)

    # Zero the gradients outside the loop
    optimizer.zero_grad()

    # Iterate through training data with a sliding window
    for i in range(0, len(x_train_tensor) - window_size, batch_size):
        # Extract the input and target sequences
        #x_batch = x_train_tensor[i:i+window_size].view(batch_size, window_size, -1)
        x_batch = x_train_tensor[i:i+window_size].view(batch_size, window_size, 7)


        y_batch = y_train_tensor[i:i+window_size]

        # Forward pass
        output, hidden = model(x_batch, hidden)

        # Detach hidden state
        hidden = (hidden[0].detach(), hidden[1].detach())

        # Compute the loss
        loss = criterion(output, y_batch)

        # Backward pass
        loss.backward()

    # Optimizer step (outside the inner loop)
    optimizer.step()

    # Print training loss periodically
    if epoch % batch_evaluation_frequency == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item()}')


RuntimeError: shape '[16, 20, 7]' is invalid for input of size 140

In [ ]:
ä

In [ ]:
# Training loop
for epoch in range(epochs):
    # Initialize hidden state for each epoch
    hidden = model.init_hidden(batch_size)

    # Zero the gradients outside the loop
    optimizer.zero_grad()

    # Iterate through training data with a sliding window
    for i in range(0, len(x_train_tensor) - window_size, batch_size):
        # Extract the input and target sequences
        x_batch = x_train_tensor[i:i+window_size].view(batch_size, window_size, -1)
        y_batch = y_train_tensor[i:i+window_size]

        # Forward pass
        output, hidden = model(x_batch, hidden)

        # Detach hidden state
        hidden = (hidden[0].detach(), hidden[1].detach())

        # Compute the loss
        loss = criterion(output, y_batch)

        # Backward pass
        loss.backward()

    # Optimizer step (outside the inner loop)
    optimizer.step()

    # Print training loss periodically
    if epoch % batch_evaluation_frequency == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item()}')


RuntimeError: shape '[50, 50, -1]' is invalid for input of size 350

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Instantiate the model
model = LSTMModel(input_size, hidden_dim, n_layers, output_size)

# Choose a loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

epochs = 1
window_size = 20

# Training loop
for epoch in range(epochs):
    for i in range(0, len(x_train) - window_size, batch_size):
        inputs = x_train[i:i+window_size]
        targets = y_train[i+window_size]

        # Convert inputs and targets to PyTorch tensors
        inputs = torch.tensor(inputs, dtype=torch.float32)
        targets = torch.tensor(targets, dtype=torch.float32)

        # Zero the gradients
        optimizer.zero_grad()

        # Initialize hidden state
        hidden = model.init_hidden(batch_size)

        # Forward pass
        outputs, _ = model(inputs.unsqueeze(0), hidden)
        loss = criterion(outputs.squeeze(), targets)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Print loss or other metrics occasionally
        if i % batch_evaluation_frequency == 0:
            print(f'Epoch {epoch}, Batch {i}, Loss: {loss.item()}')




KeyError: 20